In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

In [2]:
df = pd.read_csv('data/train.csv')

Данные представлены в формате CSV.  Каждая строка представляет молекулу.

- Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
- Остальные столбцы D1-D1776 представляют собой молекулярные **дескрипторы** — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

In [4]:
df.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать **F1-score**.

Необходимо обучить две модели: **логистическую регрессию и случайный лес**. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все **четыре метода** (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

## 1. Подготовка данных

In [5]:
X = df.drop(['Activity'], axis=1)
y = df['Activity']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

## 2. Обучение модели (параметры по умолчанию)

### 2.1. Логистическая регрессия (LogisticRegression)

In [7]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 500)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)
y_train_pred = log_reg.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.89
f1_score на тестовом наборе: 0.78


### 2.2. Случайный лес (Random Forest)

In [171]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42, max_depth=20, n_estimators = 100, min_samples_leaf = 5)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.94
f1_score на тестовом наборе: 0.80


## 3. Подбор гиперпараметров по методу GridSearchCV

### 3.1. GridSearchCV. Логистическая регрессия (LogisticRegression)

In [47]:
param_grid = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 10, 10, dtype=float))}
            
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=500), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_1.fit(X_train, y_train) 
y_train_pred = grid_search_1.predict(X_train)
y_test_pred = grid_search_1.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py",

CPU times: total: 906 ms
Wall time: 1min 41s
f1_score на тренировочном наборе: 0.82
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'C': np.float64(0.01), 'penalty': 'l2', 'solver': 'lbfgs'}


### 3.2. GridSearchCV.Случайный лес (Random Forest)

In [ ]:
param_grid = {'n_estimators': list(range(100, 300, 20)),
              'min_samples_leaf': list(range(4, 10, 1)),
              'max_depth': list(np.linspace(20, 100, 10, dtype=int))
              }

grid_search_2 = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid = param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_2.fit(X_train, y_train) 
y_train_pred = grid_search_2.predict(X_train)
y_test_pred = grid_search_2.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_2.best_params_))

CPU times: total: 9.78 s
Wall time: 17min 25s
f1_score на тренировочном наборе: 0.96
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'max_depth': np.int64(20), 'min_samples_leaf': 4, 'n_estimators': 280}


## 4. Подбор гиперпараметров по методу RandomizedSearchCV

### 4.1. RandomizedSearchCV. Логистическая регрессия

In [67]:
#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_distributions ={'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 10, 10, dtype=float))}
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=500), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 15, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
y_train_pred = random_search.predict(X_train)
y_test_pred = random_search.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", l

CPU times: total: 9.97 s
Wall time: 1min 12s
f1_score на тренировочном наборе: 0.89
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': np.float64(1.12)}


### 4.2. RandomizedSearchCV. Случайный лес (Random Forest)

In [76]:
param_distributions = {'n_estimators': list(range(100, 300, 20)),
              'min_samples_leaf': list(range(4, 10, 1)),
              'max_depth': list(np.linspace(20, 100, 10, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 15, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 
y_train_pred = random_search_forest.predict(X_train)
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 2.09 s
Wall time: 27 s
f1_score на тренировочном наборе: 0.96
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'n_estimators': 200, 'min_samples_leaf': 4, 'max_depth': np.int64(100)}


## 5. Подбор гиперпараметров по методу Hyperopt c приенением кросс-валидации

### 5.1. Hyperopt. Логистическая регрессия

In [151]:
# Пространство поиска гиперпараметров
space  = {
    'penalty': hp.choice('penalty', ['l2']),
    'solver': hp.choice('solver', ['lbfgs', 'sag']),
    'C': hp.uniform('C', 0.01, 10)
}


In [152]:
# Фиксируем random_state
random_state = 42

# Функция для оптимизации
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {
        'penalty': params['penalty'], 
        'solver': params['solver'], 
        'C': float(params['C'])
    }
  
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    # Кросс-валидация для оценки качества модели
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score  # Hyperopt минимизирует метрику, поэтому знак минус


In [153]:
%%time
# Запуск оптимизации
trials = Trials()

best = fmin(
    fn=hyperopt_lr,  
    space=space,     
    algo=tpe.suggest,  
    max_evals=50,    
    trials=trials,  
    rstate=np.random.default_rng(random_state)  
)

print("Наилучшие значения гиперпараметров:", best)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [01:22<00:00,  1.65s/trial, best loss: -0.7905922573091402]
Наилучшие значения гиперпараметров: {'C': np.float64(0.04104040887245952), 'penalty': np.int64(0), 'solver': np.int64(0)}
CPU times: total: 1.78 s
Wall time: 1min 22s


In [154]:
# рассчитаем точность для тестовой выборки
p = {'penalty': ['l2'],
    'solver': ['lbfgs', 'sag']
}

model_lr = linear_model.LogisticRegression(
    random_state=random_state, 
    penalty=p['penalty'][int(best['penalty'])],
    solver=p['solver'][int(best['solver'])],
    C=float(best['C'])
)
model_lr.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.84
f1_score на тестовом наборе: 0.79


### 5.2. Hyperopt. Случайный лес (Random Forest)

In [159]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 20),
        'min_samples_leaf': hp.quniform('min_samples_leaf', 4, 10, 1),
       'max_depth' : hp.quniform('max_depth', 20, 100, 10)
      }

In [160]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [161]:
%%time
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [00:36<00:00,  1.84s/trial, best loss: -0.8123515587949139]
Наилучшие значения гиперпараметров {'max_depth': np.float64(40.0), 'min_samples_leaf': np.float64(4.0), 'n_estimators': np.float64(100.0)}
CPU times: total: 688 ms
Wall time: 36.8 s


In [162]:
# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model_rf.fit(X_train, y_train)
y_train_pred = model_rf.predict(X_train)
y_test_pred = model_rf.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.96
f1_score на тестовом наборе: 0.81


### 6. Подбор гиперпараметров по методу Optuna с применением кросс-валидации

### 6.1. Optuna. Логистическая регрессия

In [163]:
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l2'])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'sag'])
  C = trial.suggest_uniform('C', 0.01, 10)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state)
  
  # применим  cross validation на 5 фолдах
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [172]:
%%time
# cоздаем объект исследования
# максимизируем метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

[I 2025-03-13 12:35:33,933] A new study created in memory with name: LogisticRegression
C:\Users\User\AppData\Local\Temp\ipykernel_15408\3215296696.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  C = trial.suggest_uniform('C', 0.01, 10)
[I 2025-03-13 12:35:38,177] Trial 0 finished with value: 0.7711625134869025 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 8.58724483394793}. Best is trial 0 with value: 0.7711625134869025.
C:\Users\User\AppData\Local\Temp\ipykernel_15408\3215296696.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  C = trial.suggest_uniform('C', 0.01, 10)
[I 2025-03-13 12:35:41,832] Trial 1 finished with value: 0.7711625134869025 and parameters: {'penalty': 'l2', 'solver': '

CPU times: total: 812 ms
Wall time: 44.8 s


In [165]:
# рассчитаем точность для тестовой выборки
model_lr = linear_model.LogisticRegression(**best_params, random_state=random_state)
model_lr.fit(X_train, y_train)
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.84
f1_score на тестовом наборе: 0.79


### 6.2. Optuna. Случайный лес

In [166]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 20)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 4, 10, 1)
  max_depth = trial.suggest_int('max_depth', 20, 100, 10)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  
  # применим  cross validation на 5 фолдах
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [167]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2025-03-13 12:18:18,734] A new study created in memory with name: RandomForestClassifier
C:\Users\User\AppData\Local\Temp\ipykernel_15408\688606844.py:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 20)
C:\Users\User\AppData\Local\Temp\ipykernel_15408\688606844.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 4, 10, 1)
C:\Users\User\AppData\Local\Temp\ipykernel_15408\688606844.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  max_depth = trial.suggest_int('max_depth', 20, 100, 10)
[I 2025-03-13 12:18:20,822] Trial 0 finished with value: 0.7932526635942431 and parameters: {'n_estimators': 180, 'min_samples_leaf': 10, 'max_depth': 8

CPU times: total: 922 ms
Wall time: 54.9 s


In [168]:
# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model_rf.fit(X_train, y_train)
y_train_pred = model_rf.predict(X_train)
y_test_pred = model_rf.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.96
f1_score на тестовом наборе: 0.80


## Выводы:

Выполнена задача по подбору гиперпараметров 4-мя методами: GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna

Предварительно были обучены модели с параметрами по умаолчанию (для Логистической регресси) и с параметрами [max_depth=20, n_estimators = 100, min_samples_leaf = 5'] для Случайного леса.

**Полученные метрики тестовой модели:**


- LogisticRegression: f-score = 0.78
- Random Forest: f-score = 0.78


**Метрики и затраты ресурсов после подбора гиперпараметров:**


### 1. GridSearchCV:

**1.1. LogisticRegression**

- f1_score на тестовом наборе: 0.78
- Наилучшие значения гиперпараметров: {'C': np.float64(0.01), 'penalty': 'l2', 'solver': 'lbfgs'}
- Wall time: 1min 41s

**1.2. Random Forest**

- f1_score на тестовом наборе: 0.80
- Наилучшие значения гиперпараметров: {'max_depth': np.int64(20), 'min_samples_leaf': 4, 'n_estimators': 280}
- Wall time: 17min 25s

### 2. RandomizedSearchCV:

**2.1. LogisticRegression**

- f1_score на тестовом наборе: 0.78
- Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': np.float64(1.12)}
- Wall time: 1min 12s

**2.2. Random Forest**

- f1_score на тестовом наборе: 0.80
- Наилучшие значения гиперпараметров: {'n_estimators': 200, 'min_samples_leaf': 4, 'max_depth': np.int64(100)}
- Wall time: 27 s

### 3. Hyperopt (c кросс валидацией):

**3.1. LogisticRegression**

- f1_score на тестовом наборе: 0.79
- Наилучшие значения гиперпараметров: {'C': np.float64(0.04104040887245952), 'penalty': 'l2, 'solver': 'lbfgs'}
- Wall time: 1min 22s

**3.2. Random Forest**

- f1_score на тестовом наборе: 0.81
- Наилучшие значения гиперпараметров {'max_depth': np.float64(40.0), 'min_samples_leaf': np.float64(4.0), 'n_estimators': np.float64(100.0)}
- Wall time: 36.8 s

### 4. Optuna (c кросс валидацией):

**4.1. LogisticRegression**

- f1_score на тестовом наборе: 0.79
- Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'sag', 'C': 0.04424813886614554}
- Wall time: 44.8 s

**4.2. Random Forest**

- f1_score на тестовом наборе: 0.80
- Наилучшие значения гиперпараметров {'n_estimators': 240, 'min_samples_leaf': 4, 'max_depth': 100}
- Wall time: 54.9 s


> **По всем методам подбора метрики примерно совпадают. По методам Optuna и Hyperop метрики немного лучше. Также данные методы (Optuna и Hyperop) выйгрывают по временным затратам.**